# Just an exercize


In [6]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF


In [7]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


In [8]:
class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool2d = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(
                feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.finalConv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_conn = []
        for down in self.downs:
            x = down(x)
            skip_conn.append(x)
            x = self.pool2d(x)

        x = self.bottleneck(x)
        skip_conn = skip_conn[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_co = skip_conn[idx//2]

            if x.shape != skip_co.shape:
                x = TF.resize(x, size=skip_co.shape[2:])

            concat_skip = torch.cat((skip_co, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.finalConv(x)


In [45]:
x = torch.randn((3, 1, 16, 16))
model = UNET(in_channels=1, out_channels=1)
preds = model(x)
print('prediction: {}'.format(preds))
print("original: {}".format(x))

assert preds.shape == x.shape


prediction: tensor([[[[-2.5844e-01,  2.1601e-01, -1.2646e-01,  1.8504e-01,  8.2756e-02,
            4.5464e-02, -5.8519e-01, -1.2161e-01, -6.1885e-01,  1.3694e-01,
           -8.2646e-02,  4.3301e-01, -4.7469e-01, -8.2951e-03, -2.3339e-01,
           -2.1168e-01],
          [ 2.6918e-01,  7.3964e-02,  7.4930e-02, -1.6585e-01,  6.4727e-02,
            1.0430e-02,  2.0065e-01, -1.5214e-01,  1.8906e-01,  3.6654e-01,
            4.6757e-02, -4.8872e-01, -2.1960e-01,  4.2577e-02,  4.0605e-01,
            2.1675e-01],
          [-1.2825e-01,  6.4735e-02,  1.6294e-01, -2.9810e-01, -4.6799e-02,
            1.2173e-01, -2.2323e-01, -5.8048e-01,  1.0498e-01,  9.6865e-02,
           -2.8058e-01, -2.6195e-01, -1.3593e-01, -5.2072e-01,  5.5812e-02,
           -3.2855e-01],
          [ 1.1074e-02, -2.0637e-01,  3.7011e-02,  3.5278e-01, -1.1100e-01,
           -3.7134e-02,  2.6483e-01, -1.6537e-01, -6.1749e-02,  2.0519e-01,
           -2.6593e-01,  2.3431e-01, -2.3385e-01,  2.2581e-01, -5.1204e-01,
 